In [2]:
import trackhhl.toy.simple_generator as toy
import trackhhl.event_model.q_event_model as em
import numpy as np
import itertools
import copy
import psutil
import cProfile
import time
from scipy.sparse import lil_matrix
from dimod import BinaryQuadraticModel
from neal import SimulatedAnnealingSampler
import dimod

N_MODULES = 3
LX = float("+inf")
LY = float("+inf")
Z_SPACING = 1.0

detector = toy.SimpleDetectorGeometry(
    module_id=list(range(N_MODULES)),
    lx=[LX] * N_MODULES,
    ly=[LY] * N_MODULES,
    z=[i + Z_SPACING for i in range(N_MODULES)]
)
generator = toy.SimpleGenerator(
    detector_geometry=detector,
    theta_max=np.pi / 6
)

N_PARTICLES = 2
event = generator.generate_event(N_PARTICLES)

process = psutil.Process()
profiler = cProfile.Profile()
profiler.enable()

start_time = time.time()
mem_ham1 = process.memory_info().rss / (1024 ** 2)

def generate_hamiltonian(event, params):
    lambda_val = params.get('lambda')
    alpha = params.get('alpha')
    beta = params.get('beta')

    # Use deep copy to match Algorithm 1
    modules = copy.deepcopy(event.modules)
    modules.sort(key=lambda a: a.z)

    # Eagerly generate all segments like in Algorithm 1
    segments = [
        em.segment(from_hit, to_hit)
        for idx in range(len(modules) - 1)
        for from_hit, to_hit in itertools.product(modules[idx].hits, modules[idx + 1].hits)
    ]

    N = len(segments)
    A_ang = lil_matrix((N, N), dtype=np.float32)
    A_bif = lil_matrix((N, N), dtype=np.float32)
    A_inh = lil_matrix((N, N), dtype=np.float32)

    # Precompute the segment relationships
    s_ab = np.zeros((N, N))
    for i, seg_i in enumerate(segments):
        for j, seg_j in enumerate(segments):
            s_ab[i, j] = int(seg_i.from_hit.module_id == 1 and seg_j.to_hit.module_id == 1)

    # Caching norms for vectors like in Algorithm 2 but matching behavior of Algorithm 1
    norm_cache = {}
    def get_vector_and_norm(seg, idx):
        if idx not in norm_cache:
            vect = seg.to_vect()
            norm_cache[idx] = (vect, np.linalg.norm(vect))
        return norm_cache[idx]

    eps = 1e-9

    # Matching the logic of Algorithm 1 for filling the matrices
    for i, seg_i in enumerate(segments):
        vect_i, norm_i = get_vector_and_norm(seg_i, i)
        
        for j, seg_j in enumerate(segments):
            if i != j:
                vect_j, norm_j = get_vector_and_norm(seg_j, j)

                cosine = np.dot(vect_i, vect_j) / (norm_i * norm_j)

                if np.abs(cosine - 1) < eps:
                    A_ang[i, j] = 1

                if (seg_i.from_hit == seg_j.from_hit) and (seg_i.to_hit != seg_j.to_hit):
                    A_bif[i, j] = -alpha

                if (seg_i.from_hit != seg_j.from_hit) and (seg_i.to_hit == seg_j.to_hit):
                    A_bif[i, j] = -alpha

                A_inh[i, j] = s_ab[i, j] * s_ab[j, i] * beta

    # Compute the final expression like in Algorithm 1
    A = -1 * (A_ang + A_bif + A_inh).tocsc() 
    b = np.zeros(N, dtype=np.float32)

    components = {
        'A_ang': -A_ang,
        'A_bif': -A_bif,
        'A_inh': -A_inh,
    }

    return A, b, components, segments

# Parameters
params = {
    'alpha': 1.0,
    'beta': 1.0,
    'lambda': 100.0,
}

# Generate the Hamiltonian
A, b, _, _ = generate_hamiltonian(event, params)

finish_time = time.time()
mem_ham2 = process.memory_info().rss / (1024 ** 2)
profiler.disable()
profiler.print_stats()

print(A)
print(f'{finish_time - start_time}s')
print(f'{(mem_ham2 - mem_ham1):.6f}gb')

# Solving the QUBO problem (same as in Algorithm 2 but matching the structure of Algorithm 1)
from scipy.sparse import csr_matrix

process = psutil.Process()
start_time = time.time()

initial_memory = process.memory_info().rss / (1024 ** 2)

# Convert A to a sparse CSR matrix for efficiency
matrix_A = csr_matrix(A)

# Create BinaryQuadraticModel
bqm = BinaryQuadraticModel({}, {}, 0.0, dimod.BINARY)

# Add variables and interactions
for i in range(len(b)):
    bqm.add_variable(i, b[i])

for i, j in zip(*matrix_A.nonzero()):
    bqm.add_interaction(i, j, matrix_A[i, j])

# Convert to QUBO
q, offset = bqm.to_qubo()

# Simulated annealing
sampler = SimulatedAnnealingSampler()
response = sampler.sample_qubo(q, num_reads=100)

# Best solution
best_sample = response.first.sample
sol_sample = np.fromiter(best_sample.values(), dtype=int)  # Efficient conversion to numpy array

end_time = time.time()
final_memory = process.memory_info().rss / (1024 ** 2)
print(sol_sample)
print(f'Execution time: {end_time - start_time:.6f} seconds')
print(f'Memory usage: {final_memory - initial_memory:.6f} MB')


         3789 function calls (3643 primitive calls) in 0.016 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 1673254091.py:1(<module>)
        1    0.002    0.002    0.011    0.011 1673254091.py:40(generate_hamiltonian)
        3    0.000    0.000    0.000    0.000 1673254091.py:47(<lambda>)
        1    0.000    0.000    0.000    0.000 1673254091.py:50(<listcomp>)
       64    0.000    0.000    0.000    0.000 1673254091.py:69(get_vector_and_norm)
        2    0.000    0.000    0.000    0.000 <frozen abc>:117(__instancecheck__)
       38    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:405(parent)
        2    0.000    0.000    0.000    0.000 <string>:1(<lambda>)
        8    0.000    0.000    0.000    0.000 <string>:2(__init__)
        2    0.000    0.000    0.000    0.000 __init__.py:1052(memory_info)
       18    0.000    0.000    0.000    0.000 _base.py:108(